# IMPORTING LIBRARIES

In [79]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
from pathlib import Path
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import cm as cmaps
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from dateutil.relativedelta import relativedelta
from collections import Counter, OrderedDict
import itertools
import re

print("Packages loaded!")

Packages loaded!


# Pipeline Controller

hk_df=load_original_data() 
hk_df=lower_case_column_names(hk_df)
##hk_df=rename_columns(hk_df)
#hk_df=drop_columns(hk_df)

hk_df=load_original_data() 

hk_df= (hk_df.pipe(lower_case_column_names)\
.pipe(rename_columns)
 #.pipe(drop_columns)



# READ DATA INTO DATAFRAME FROM EXISTING .CSV

In [80]:
def load_original_data():
    """
    Read production data and parse into pandas dataframe.
    """
    latest_file = r"C:\Users\aykut\OneDrive\Belgeler\DA_Midterm_Project\DA_Midterm_Project\Data\Production_Data.csv"
    df_prod_whole = pd.read_csv(latest_file, sep=";", encoding = "ISO-8859-1")
    
    return df_prod_whole

df_prod_whole = load_original_data()

# DF CLEANING

## Delete unnecessary/empty colums

In [81]:
df_prod_whole.columns.str.contains("WT1")

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [82]:
def delete_unncessary_columns(df):
    """
    Delete columns only containing NaN values or columns with effectively useless data
    """
    #try:
    cols_to_drop = []
    if any(df_prod_whole.columns.str.contains("WT1")):
        cols_to_drop_1 = [col for col in df_prod_whole.columns if "WT1" in col]
        cols_to_drop.extend(cols_to_drop_1)
    if any(df_prod_whole.columns.str.contains("WT2")):
        cols_to_drop_2 = [col for col in df_prod_whole.columns if "WT2" in col]
        cols_to_drop.extend(cols_to_drop_2)
    if any(df_prod_whole.columns.str.contains("ET2")):
        cols_to_drop_3 = [col for col in df_prod_whole.columns if "ET2" in col]
        cols_to_drop.extend(cols_to_drop_3)
    if any(df_prod_whole.columns.str.contains("FT2")):
        cols_to_drop_4 = [col for col in df_prod_whole.columns if "FT2" in col]
        cols_to_drop.extend(cols_to_drop_4)
    if any(df_prod_whole.columns.str.contains("Schneiden_Ort")):
        cols_to_drop_5 = [col for col in df_prod_whole.columns if "Schneiden_Ort" in col]
        cols_to_drop.extend(cols_to_drop_5)
    if any(df_prod_whole.columns.str.contains("Schneiden_Geraet")):
        cols_to_drop_6 = [col for col in df_prod_whole.columns if "Schneiden_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_6)
    if any(df_prod_whole.columns.str.contains("Schleifen_Ort")):
        cols_to_drop_7 = [col for col in df_prod_whole.columns if "Schleifen_Ort" in col]
        cols_to_drop.extend(cols_to_drop_7)
    if any(df_prod_whole.columns.str.contains("Schleifen_Geraet")):
        cols_to_drop_8 = [col for col in df_prod_whole.columns if "Schleifen_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_8)
    if any(df_prod_whole.columns.str.contains("NS2_Ort")):
        cols_to_drop_9 = [col for col in df_prod_whole.columns if "NS2_Ort" in col]
        cols_to_drop.extend(cols_to_drop_9)
    if any(df_prod_whole.columns.str.contains("OeO_Ort")):
        cols_to_drop_10 = [col for col in df_prod_whole.columns if "OeO_Ort" in col]
        cols_to_drop.extend(cols_to_drop_10)
    if any(df_prod_whole.columns.str.contains("OeO_Geraet")):
        cols_to_drop_11 = [col for col in df_prod_whole.columns if "OeO_Geraet" in col]
        cols_to_drop.extend(cols_to_drop_11)
        
    else:
        raise ValueError("Columns have already been deleted!")

    cols_to_drop.extend(["Versand_Intern", "SSMA_TimeStamp", "Assembley_Teflonschlauch-Charge"])
    df.drop(cols_to_drop, axis=1, inplace=True)

    #except:
        #raise ValueError("Deleting columns was unsuccessful!")
    
    return df

df_cleaned = delete_unncessary_columns(df_prod_whole)

## Make columns lowercase and fix syntax

In [83]:
def make_columns_lowercase(df):
    
    lower_cols = [i.lower().replace(" ", "_").replace("-", "_") for i in df.columns]
    df_cleaned.columns = lower_cols    
    
    return df_cleaned

df_cleaned = make_columns_lowercase(df_cleaned)

## Remove rows with XDC-ID < `68.000`

In [84]:
def delete_rows_with_XDC_ID_pre_68k(df):
    """
    Delete all entries before XDC-ID 68.000 due to them having wrong defect IDs.
    Only applies if dataframe actually has older entries with XDC-IDs < 68.000
    """
    
    df = df[df["pdc_nummer"]>=68000]
    
    return df

#df_cleaned = delete_rows_with_XDC_ID_pre_68k(df_cleaned)

## Remove rows with non-legit XDC-ID

In [85]:
def delete_non_legit_XDC_IDs(df):
    """
    Delete all entries before XDC-ID 68.000 due to them having wrong defect IDs.
    Only applies if dataframe actually has older entries with XDC-IDs < 68.000
    """
    
    df = df[df["pdc_nummer"]>=44145]
    
    return df

df_cleaned = delete_non_legit_XDC_IDs(df_cleaned)

## Cast `string-dates` into `datetime-dates`

In [86]:
def date_columns_transform_into_datetime(df):
    """
    Transform all dates of string type 'YYYY-MM-DD' into dt.datetime format.
    """
    date_cols = [col for col in df.columns if "datum" in col]

    for col in date_cols:
        if df[col].dtype == "object":
            df[col] = [dt.datetime.strptime(d,'%Y-%m-%d').date() if type(d)==str 
                       else np.nan if type(d)==float
                       else d
                       for d in df[col]]
            
    return df, date_cols

df_cleaned, date_columns = date_columns_transform_into_datetime(df_cleaned)

## Remove rows without entries in selected columns

In [87]:
def delete_rows_with_nan_in_selected_cols(df, cols):
    """
    Delete rows that do not have an entry in the given columns
    """
    for col in cols:
        df[col].dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        
    return df
    
df_cleaned = delete_rows_with_nan_in_selected_cols(df_cleaned, ["schmelzen_datum", "ns1_datum"])

## Substitute Product-`Familie` int-values with strings *("PDC", "PDC-C", "XDC")*

In [88]:
def substitute_familie_int_str(df):
    """
    Parse int values (1,2,3) into string values ("pdc", "pdc-c", "xdc")
    """
    
    if df["familie"].dtype == "int64":
        df["familie"] = ["pdc" if fam==1
                         else "pdc-c" if fam==2
                         else "xdc" 
                         for fam in df_cleaned["familie"]]
        
    return df

df_cleaned = substitute_familie_int_str(df_cleaned)

## Create `_Months` columns

In [89]:
def add_date_month_string_columns(df, date_columns):
    """
    Convert YYYY-MM-DD dt.datetime into new 'YYYY-MM' string month column. 
    """
    
    unwanted_dates = ['ts_in_datum', 'ts_out_datum', 'rt_bqc_in_datum', 'rt_bqc_out_datum']
    date_columns_wanted = [col for col in date_columns if col not in unwanted_dates]
    date_columns_months = [col.split("_")[0] + "_month" for col in date_columns_wanted]
    #date_columns_months = [col  for col in date_columns_months]
    
    #if "schmelzen_Mmnths" in df.columns:
    for idx, datum in enumerate(date_columns_wanted):
        df[date_columns_months[idx]] = [str(d)[:-3] for d in df[datum]]
    #else:
        #raise ValueError("Months columns have already been added!")
    return df

df_cleaned = add_date_month_string_columns(df_cleaned, date_columns)

### -> To Fix - Aykut

- fix Personal-entries so that only uppercase letters make up entries
    - no symbols like "_", "/", or multiple entries

> numerical features that needs to be standardized
* ns1, ns2, oeo, bqc, et1, ft1, ek

In [90]:
list(df_cleaned.columns)

['pdc_nummer',
 'produktion_groesse',
 'familie',
 'art',
 'kommentar_produktion',
 'grenzwert_set',
 'defekt_arbeitsschritt_id',
 'defekt_grund_id',
 'schmelzen_personal',
 'schmelzen_datum',
 'schmelzen_ort',
 'schmelzen_geraet',
 'schmelzen_kapillaren_charge',
 'ns1_personal',
 'ns1_datum',
 'ns1_ort',
 'ns1_geraet',
 'ns1_winkel_vorne',
 'ns1_winkel_hinten',
 'ns1_durchmesser_vorne',
 'ns1_durchmesser_hinten',
 'schneiden_personal',
 'schneiden_datum',
 'schleifen_personal',
 'schleifen_datum',
 'ns2_personal',
 'ns2_datum',
 'ns2_geraet',
 'ns2_center_variation',
 'ns2_front_angle',
 'ns2_curve',
 'ns2_15xlength',
 'ns2_parabolic_factor',
 'oeo_personal',
 'oeo_datum',
 'oeo_oeo',
 'kleben1_personal',
 'kleben1_datum',
 'kleben1_ort',
 'kleben1_geraet',
 'kleben1_kleber_charge',
 'kleben1_kleber_teilentnahme',
 'kleben1_piezzo_charge',
 'kleben2_personal',
 'kleben2_datum',
 'kleben2_ort',
 'kleben2_geraet',
 'kleben2_kleber_charge',
 'kleben2_kleber_teilentnahme',
 'kleben2_huels

In [92]:
def commatodot(df):     
    columns = ['ns1_winkel_vorne','ns1_winkel_hinten','ns1_durchmesser_vorne','ns1_durchmesser_hinten',
               "ns2_center_variation", "ns2_front_angle","ns2_curve","ns2_15xlength",
               "ns2_parabolic_factor","oeo_oeo", "piezo_phase"] 
                                                            #replacing commas with dots in numbers and keeping string values
    for column in columns:
        df[column] = [float(str(x).replace(',','.')) for x in df[column]] 
     
    return df
                    

In [93]:
commatodot(df_cleaned)

,pdc_nummer,produktion_groesse,familie,art,kommentar_produktion,grenzwert_set,defekt_arbeitsschritt_id,defekt_grund_id,schmelzen_personal,schmelzen_datum,schmelzen_ort,schmelzen_geraet,schmelzen_kapillaren_charge,ns1_personal,ns1_datum,ns1_ort,ns1_geraet,ns1_winkel_vorne,ns1_winkel_hinten,ns1_durchmesser_vorne,ns1_durchmesser_hinten,schneiden_personal,schneiden_datum,schleifen_personal,schleifen_datum,ns2_personal,ns2_datum,ns2_geraet,ns2_center_variation,ns2_front_angle,ns2_curve,ns2_15xlength,ns2_parabolic_factor,oeo_personal,oeo_datum,oeo_oeo,kleben1_personal,kleben1_datum,kleben1_ort,kleben1_geraet,kleben1_kleber_charge,kleben1_kleber_teilentnahme,kleben1_piezzo_charge,kleben2_personal,kleben2_datum,kleben2_ort,kleben2_geraet,kleben2_kleber_charge,kleben2_kleber_teilentnahme,kleben2_huelse1_charge,kleben2_isolator_charge,kleben2_huelse2_charge,assembley_personal,assembley_datum,assembley_ort,assembley_huelse3_charge,uebergabe_datum_qc,dose,bqc_personal,bqc_datum,bqc_ort,bqc_geraet,bqc_spannung,bqc_puls,bqc_volumen,bqc_auslenkung,bqc_z_offset,bqc_kommentar,ts_in_datum,ts_out_datum,rt_bqc_in_datum,rt_bqc_out_datum,uebergabe_ss,et1_datum,et1_personal,et1_spannung,et1_puls,et1_volumen,et1_auslenkung,et1_z_offset,et1_ort,et1_geraet,et1_kopfid,et1_kanal,et1_version,et1_kommentar,coatingtyp,coatingtyp_personal,ft1_typ,ft1_datum,ft1_personal,ft1_spannung,ft1_puls,ft1_volumen,ft1_auslenkung,ft1_z_offset,ft1_ort,ft1_geraet,ft1_kopfid,ft1_kanal,ft1_version,ft1_kommentar,ek_datum,ek_personal,ek_spannung,ek_puls,ek_volumen,ek_groesse,ek_auslenkung,ek_z_offset,ek_kommentar,ek_ort,ek_geraet,ek_kopfid,ek_kanal,ek_version,versand_uebergabe_datum,versand_uebergabe_personal,versand_kunde,versand_order_nr,versand_datum,versand_personal,id_aktuelles_coating,piezo_position,piezo_phase,piezo_id,ns2_linear_factor,kleben1_shore,bqc_kopfid,bqc_kanal,bqc_version,schmelzen_month,ns1_month,schneiden_month,schleifen_month,ns2_month,oeo_month,kleben1_month,kleben2_month,assembley_month,uebergabe_month,bqc_month,et1_month,ft1_month,ek_month,versand_month
0,168284,1.00,pdc,1,FH Kammer 1+3 v=68.h=58 PF hoch,1,6.00,10.00,RAP,2022-08-29,Berlin,5.00,133,CT,2022-08-30,Berlin,1.00,8.00,16.00,70.10,57.40,CT,2022-08-30,FT,2022-09-05,GT,2022-09-05,1.00,0.15,0.25,1.40,5.78,35.20,DY,2022-09-05,0.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,1.00,1.00,NaN,NaN,NaN,1.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,"-0,01267",NaN,NaN,NaN,NaN,2022-08,2022-08,2022-08,2022-09,2022-09,2022-09,,,,,,,,,
1,168283,1.00,pdc,1,FH Kammer 1+3 v=68.h=58 PF hoch,1,1.00,1.00,RAP,2022-08-29,Berlin,5.00,133,CT,2022-08-30,Berlin,1.00,9.00,15.00,70.00,60.20,CT,2022-08-30,FT,2022-09-05,GT,2022-09-05,1.00,0.36,0.25,1.30,5.35,33.00,DY,2022-09-05,53.35,NaN,NaN,NaN,NaN,1.00,0.00,1.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,1.00,1.00,NaN,NaN,NaN,1.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,"0,02313",NaN,NaN,NaN,NaN,2022-08,2022-08,2022-08,2022-09,2022-09,2022-09,,,,,,,,,
2,168282,1.00,pdc,1,FH Kammer 1+3 v=68.h=58 PF hoch,1,1.00,1.00,RAP,2022-08-29,Berlin,5.00,133,CT,2022-08-30,Berlin,1.00,17.00,31.00,69.90,59.60,CT,2022-08-30,FT,2022-09-05,GT,2022-09-05,1.00,0.43,0.68,1.10,5.46,33.40,DY,2022-09-05,53.35,NaN,NaN,NaN,NaN,1.00,0.00,1.00,NaN,NaN,NaN,NaN,1.00,0.00,1.00,1.00,1.00,NaN,NaN,NaN,1.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [78]:
for element in df_cleaned["ns2_center_variation"]:
    print(element, type(element))

0.154 <class 'float'>
0.36 <class 'float'>
0.426 <class 'float'>
0.385 <class 'float'>
0.0 <class 'float'>
0.604 <class 'float'>
0.589 <class 'float'>
0.528 <class 'float'>
0.612 <class 'float'>
0.309 <class 'float'>
0.278 <class 'float'>
0.454 <class 'float'>
0.0 <class 'float'>
0.807 <class 'float'>
0.23 <class 'float'>
0.234 <class 'float'>
0.483 <class 'float'>
0.496 <class 'float'>
0.0 <class 'float'>
1.139 <class 'float'>
0.192 <class 'float'>
0.88 <class 'float'>
0.962 <class 'float'>
0.652 <class 'float'>
0.303 <class 'float'>
0.908 <class 'float'>
0.002 <class 'float'>
0.0 <class 'float'>
0.386 <class 'float'>
0.0 <class 'float'>
0.964 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.839 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.118 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.314 <class 'float'>
0.188 <class 'float'>
0.819 <class 'float'>
0.304 <class 'float'>
0.863 <class 'float'>
1.206 <class 'float'>
0.764 <class 'f

0.0 <class 'float'>
0.222 <class 'float'>
0.0 <class 'float'>
0.128 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.121 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.145 <class 'float'>
0.23 <class 'float'>
0.27 <class 'float'>
0.0 <class 'float'>
0.033 <class 'float'>
0.049 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.77 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.705 <class 'float'>
0.36 <class 'float'>
0.0 <class 'float'>
0.945 <class 'float'>
0.753 <class 'float'>
0.026 <class 'float'>
0.0 <class 'float'>
1.159 <class 'float'>
0.688 <class 'float'>
0.015 <class 'float'>
0.0 <class 'float'>
0.664 <class 'float'>
0.0 <class 'float'>
0.826 <class 'float'>
0.0 <class 'float'>
0.113 <class 'float'>
0.532 <class 'float'>
0.757 <class 'float'>
0.4 <class 'float'>
0.355 <class 'float'>
0.646 <class 'float'>
0.453 <class 'float'>
0.053 <class 'float'>
0.327 <class 'float'>
0.1 <class

0.805 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.132 <class 'float'>
0.0 <class 'float'>
0.819 <class 'float'>
0.635 <class 'float'>
0.23 <class 'float'>
0.003 <class 'float'>
0.0 <class 'float'>
0.75 <class 'float'>
0.0 <class 'float'>
0.684 <class 'float'>
0.429 <class 'float'>
0.871 <class 'float'>
0.607 <class 'float'>
0.0 <class 'float'>
0.23 <class 'float'>
0.0 <class 'float'>
0.337 <class 'float'>
0.0 <class 'float'>
0.541 <class 'float'>
0.409 <class 'float'>
0.826 <class 'float'>
1.121 <class 'float'>
0.0 <class 'float'>
1.024 <class 'float'>
0.003 <class 'float'>
0.78 <class 'float'>
0.313 <class 'float'>
1.321 <class 'float'>
1.237 <class 'float'>
0.768 <class 'float'>
0.323 <class 'float'>
0.503 <class 'float'>
0.648 <class 'float'>
0.011 <class 'float'>
0.196 <class 'float'>
0.002 <class 'float'>
0.0 <class 'float'>
1.046 <class 'float'>
0.362 <class 'float'>
0.196 <class 'float'>
0.0 <class 'float'>
0.002 <class 'float'>
0.0 <class 'floa

0.1 <class 'float'>
0.121 <class 'float'>
0.052 <class 'float'>
0.3 <class 'float'>
0.0 <class 'float'>
0.735 <class 'float'>
0.048 <class 'float'>
0.816 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.728 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.074 <class 'float'>
0.324 <class 'float'>
1.048 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.955 <class 'float'>
0.187 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.739 <class 'float'>
0.0 <class 'float'>
1.505 <class 'float'>
0.137 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.45 <class 'float'>
0.362 <class 'float'>
0.952 <class 'float'>
0.512 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.325 <class 'float'>
0.945 <class 'float'>
0.388 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.733 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.565 <class 'float'>
0.0 <class 'float'>
0.304 <class 'f

0.255 <class 'float'>
0.0 <class 'float'>
0.333 <class 'float'>
0.716 <class 'float'>
0.071 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.003 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.003 <class 'float'>
0.57 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.104 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.475 <class 'float'>
0.248 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.187 <class 'float'>
0.367 <class 'float'>
0.562 <class 'float'>
0.0 <class 'float'>
0.397 <class 'float'>
0.693 <class 'float'>
0.131 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.377 <class 'float'>
0.001 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.219 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.01 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.007 <class 'float'>
0.259 <class 'float'>
0.873 <class 'float'>
0.0 <class '

...

0.024 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.014 <class 'float'>
0.016 <class 'float'>
0.019 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.019 <class 'float'>
0.009 <class 'float'>
0.027 <class 'float'>
0.014 <class 'float'>
0.014 <class 'float'>
0.036 <class 'float'>
0.0 <class 'float'>
0.012 <class 'float'>
0.0 <class 'float'>
0.019 <class 'float'>
0.01 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.007 <class 'float'>
0.011 <class 'float'>
0.007 <class 'float'>
0.0 <class 'float'>
0.027 <class 'float'>
0.011 <class 'float'>
0.012 <class 'float'>
0.0 <class 'float'>
0.006 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.015 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.0 <class 'float'>
0.0 <class 'flo

0.0 <class 'float'>
0.0 <class 'float'>
0.005 <class 'float'>
0.014 <class 'float'>
0.0 <class 'float'>
0.009 <class 'float'>
0.006 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.019 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.036 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.02 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.013 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.016 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.024 <class 'float'>
0.018 <class 'float'>
0.0 <class 'float'>
0.01 <class 'fl

0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.011 <class 'float'>
0.013 <class 'float'>
0.011 <class 'float'>
0.004 <class 'float'>
0.006 <class 'float'>
0.019 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.006 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.014 <class 'float'>
0.02 <class 'float'>
0.0 <class 'float'>
0.013 <class 'float'>
0.01 <class 'float'>
0.017 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.029 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.007 <class 'float'>
0.009 <class 'float'>
0.031 <class 'float'>
0.017 <class 'float'>
0.029 <class 'float'>
0.014 <class 'float'>
0.004 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.008 <class 'float'>
0.007 <class 'float'>
0.0 <class 'float'>
0.023 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.012 <class 'float'>
0.019 <class

0.0 <class 'float'>
0.0 <class 'float'>
0.024 <class 'float'>
0.0 <class 'float'>
0.017 <class 'float'>
0.0 <class 'float'>
0.002 <class 'float'>
0.021 <class 'float'>
0.0 <class 'float'>
0.01 <class 'float'>
0.0 <class 'float'>
0.014 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.0 <class 'float'>
0.026 <class 'float'>
0.012 <class 'float'>
0.0 <class 'float'>
0.001 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.012 <class 'float'>
0.013 <class 'float'>
0.014 <class 'float'>
0.038 <class 'float'>
0.038 <class 'float'>
0.0 <class 'float'>
0.055 <class 'float'>
0.013 <class 'float'>
0.029 <class 'float'>
0.027 <class 'float'>
0.011 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.017 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.028 <class 'float'>
0.0 <class 'float'>
0.018 <class 'float'>
0.0 <class 'float'>
0.0 <class 'f

0.0 <class 'float'>
0.0 <class 'float'>
0.015 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.0 <class 'float'>
0.035 <class 'float'>
0.0 <class 'float'>
0.014 <class 'float'>
0.0 <class 'float'>
0.006 <class 'float'>
0.014 <class 'float'>
0.0 <class 'float'>
0.022 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.007 <class 'float'>
0.003 <class 'float'>
0.013 <class 'float'>
0.007 <class 'float'>
0.0 <class 'float'>
0.021 <class 'float'>
0.014 <class 'float'>
0.016 <class 'float'>
0.009 <class 'float'>
0.038 <class 'float'>
0.005 <class 'float'>
0.018 <class 'float'>
0.015 <class 'float'>
0.008 <class 'float'>
0.02 <class 'float'>
0.007 <class 'float'>
0.0 <class 'float'>
0.021 <class 'float'>
0.001 <class 'float'>
0.016 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.026 <class 'float'>
0.014 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.008 <class 'float'>
0.028 <class 'float'>
0.0 <class 'float'>
0.0 <class 'float'>
0.0